In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!gdown --fuzzy https://drive.google.com/file/d/13Af9acvGqDwXwBVQYcJfUeFkItVEPmlJ/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=13Af9acvGqDwXwBVQYcJfUeFkItVEPmlJ
To: /content/lab5_data.zip
100% 18.4M/18.4M [00:01<00:00, 14.1MB/s]


In [3]:
!unzip lab5_data.zip

Archive:  lab5_data.zip
   creating: data/
   creating: data/test/
   creating: data/test/African Elephant/
  inflating: data/test/African Elephant/351.jpg  
  inflating: data/test/African Elephant/353.jpg  
  inflating: data/test/African Elephant/354.jpg  
  inflating: data/test/African Elephant/355.jpg  
  inflating: data/test/African Elephant/356.jpg  
  inflating: data/test/African Elephant/358.jpg  
  inflating: data/test/African Elephant/359.jpg  
  inflating: data/test/African Elephant/360.jpg  
  inflating: data/test/African Elephant/361.jpg  
  inflating: data/test/African Elephant/362.jpg  
  inflating: data/test/African Elephant/363.jpg  
  inflating: data/test/African Elephant/364.jpg  
  inflating: data/test/African Elephant/365.jpg  
  inflating: data/test/African Elephant/367.jpg  
  inflating: data/test/African Elephant/368.jpg  
  inflating: data/test/African Elephant/369.jpg  
  inflating: data/test/African Elephant/370.jpg  
  inflating: data/test/African Elephant/37

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
        rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(256, 256),
    batch_size=32
)
validation_generator = val_datagen.flow_from_directory(
    'data/test',
    target_size=(256, 256),
    batch_size=32
)

Found 1095 images belonging to 3 classes.
Found 274 images belonging to 3 classes.


In [6]:
num_classes=train_generator.num_classes
nb_train_samples = train_generator.samples
nb_val_samples = validation_generator.samples

In [7]:
from tensorflow.keras.applications.vgg19 import VGG19

In [8]:
base_model = VGG19(include_top=False, weights='imagenet', input_tensor=None, input_shape=(256, 256, 3))

80134624/80134624 [==============================] - 0s 0us/step


In [9]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [10]:
from tensorflow.keras.layers import Flatten, Dense, Softmax
from tensorflow.keras import Model
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [28]:
base_model.output

<KerasTensor: shape=(None, 8, 8, 512) dtype=float32 (created by layer 'block5_pool')>

In [29]:
def prepare_VGG_model_for_finetuning(freeze_baselayers=True):
  
    base_model = VGG19(include_top=False, weights='imagenet', input_tensor=None, input_shape=(256, 256, 3))    # freeze layers
    if freeze_baselayers==True:
        for layer in base_model.layers:
            layer.trainable=False
    # change here - take the output from different layers of the base model. Eg. base_model.output, base_model.layers[11].output
    x =  base_model.output
    x = Flatten()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(32, activation="relu")(x)
    x = Dense(num_classes)(x)
    prediction_layer = Softmax()(x)

    model_new = Model(inputs=base_model.input, outputs=prediction_layer)
    return model_new

In [30]:
model = prepare_VGG_model_for_finetuning(freeze_baselayers=True)


model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0   

In [20]:
model.get_layer('block5_conv1').trainable = True
model.get_layer('block5_conv2').trainable = True
model.get_layer('block5_conv3').trainable = True
model.get_layer('block5_conv4').trainable = True

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0   

In [31]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

optimizer_fn = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer_fn, loss=loss_fn, metrics=['accuracy'])

In [32]:
history = model.fit(train_generator, steps_per_epoch=nb_train_samples//32, epochs=20, validation_data=validation_generator, validation_steps=nb_val_samples//32)

Epoch 1/20
34/34 [==============================] - 19s 539ms/step - loss: 0.8942 - accuracy: 0.6378 - val_loss: 0.8377 - val_accuracy: 0.6953
Epoch 2/20
34/34 [==============================] - 18s 534ms/step - loss: 0.8228 - accuracy: 0.7178 - val_loss: 0.8186 - val_accuracy: 0.7188
Epoch 3/20
34/34 [==============================] - 18s 537ms/step - loss: 0.8173 - accuracy: 0.7225 - val_loss: 0.8190 - val_accuracy: 0.7188
Epoch 4/20
34/34 [==============================] - 18s 530ms/step - loss: 0.8124 - accuracy: 0.7272 - val_loss: 0.8080 - val_accuracy: 0.7383
Epoch 5/20
34/34 [==============================] - 18s 531ms/step - loss: 0.6937 - accuracy: 0.8495 - val_loss: 0.6314 - val_accuracy: 0.9336
Epoch 6/20
34/34 [==============================] - 18s 550ms/step - loss: 0.5986 - accuracy: 0.9520 - val_loss: 0.6271 - val_accuracy: 0.9141
Epoch 7/20
34/34 [==============================] - 18s 530ms/step - loss: 0.5837 - accuracy: 0.9699 - val_loss: 0.6047 - val_accuracy: 0.9492

## We used Transfer learning by transfering the feature extraction layers(Convolutional layers) from VGG19 model trained on ImageNet dataset to classify our animal image dataset(of 3 classes/animals). The final accuracy that we got was 95.31% which is great!

Side note: Sorry for the late submission, college is hard